In [1]:
!pip install boto3
!pip install langchain
!pip install langchain_aws

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 kB 2.2 MB/s eta 0:00:00


# Running an AWS-Hosted LLM with LangChain

In this demo notebook, we demonstrate how to use the boto3 Python SDK along with the abstractions available through the LangChain package to work with Amazon Bedrock Foundation Models.

In [2]:
import boto3
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock

In [18]:
AWS_DEFAULT_REGION="..."
AWS_ACCESS_KEY_ID="..."
AWS_SECRET_ACCESS_KEY="..."
AWS_SESSION_TOKEN="..."

## Listing available models

Here we can see which foundation models Bedrock as access to.  However, remember that not all of these are active for this workshop.

In [6]:
[models['modelId'] for models in boto3_bedrock.list_foundation_models()['modelSummaries']]

['amazon.titan-tg1-large',
 'amazon.titan-embed-g1-text-02',
 'amazon.titan-text-lite-v1:0:4k',
 'amazon.titan-text-lite-v1',
 'amazon.titan-text-express-v1:0:8k',
 'amazon.titan-text-express-v1',
 'amazon.nova-pro-v1:0',
 'amazon.nova-lite-v1:0',
 'amazon.nova-micro-v1:0',
 'amazon.titan-embed-text-v1:2:8k',
 'amazon.titan-embed-text-v1',
 'amazon.titan-embed-text-v2:0',
 'amazon.titan-embed-image-v1:0',
 'amazon.titan-embed-image-v1',
 'amazon.titan-image-generator-v1:0',
 'amazon.titan-image-generator-v1',
 'amazon.titan-image-generator-v2:0',
 'amazon.rerank-v1:0',
 'stability.stable-diffusion-xl-v1:0',
 'stability.stable-diffusion-xl-v1',
 'stability.sd3-large-v1:0',
 'stability.sd3-5-large-v1:0',
 'stability.stable-image-core-v1:0',
 'stability.stable-image-core-v1:1',
 'stability.stable-image-ultra-v1:0',
 'stability.stable-image-ultra-v1:1',
 'anthropic.claude-3-5-sonnet-20241022-v2:0:18k',
 'anthropic.claude-3-5-sonnet-20241022-v2:0:51k',
 'anthropic.claude-3-5-sonnet-20241022

## Creating Bedrock runtime client

In this next code block, a dedicated boto3 client for the bedrock-runtime service is created.  This client is responsible for executing runtime operations, such as invoking a model with a given prompt.

In [7]:
bedrock_runtime = boto3.client(
    'bedrock-runtime',
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN
)

## Initializing BedrockLLM and invoking a model

Here, the BedrockLLM class from the langchain_aws package is instantiated.  This class serves as a high-level wrapper to interface with AWS-hosted LLMs.
The initialization parameters include the model ID (in this case, "amazon.titan-tg1-large"), region, and the necessary AWS credentials.  Once the instance is created, the invoke method is used to send a prompt ("What is the recipe of mayonnaise?") to the model.  This section demonstrates the fundamental workflow: setting up the model wrapper and making a basic invocation call to test the model’s response, providing a concrete example of how to interact with AWS-hosted generative AI models using LangChain.

Try several different prompts here to see what different types of answers you can get!

In [17]:
llm = BedrockLLM(
    model_id="amazon.titan-tg1-large",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN
)

llm.invoke(input='What is the recipe of mayonnaise?')

'\n1. Mayonnaise is made by combining oil, egg yolk, and an acid, either lemon juice or vinegar, in a process known as emulsification. The oil and egg yolk are initially whisked together to create a thick and creamy mixture.\n2. The acid is then gradually added to the mixture while whisking continuously. This causes the oil and egg yolk to combine and form an emulsion, which is a mixture of two liquids that do not normally mix, such as oil and water.\n3. The mayonnaise can be customized by adding additional ingredients such as mustard, herbs, or spices. It'

## Introducing ChatBedrock

BedrockLLM is designed for single-turn, prompt-based interactions where you provide the prompt ("What is the recipe for mayonnaise?") and the model generates an output in one go.  This is fine for simple things, but when you need to have more sophisticated interactions you want something that supports chat-like exchanges where the model can manage context over several turns of dialogue.  Additionally, not all of the available models, including more sophisticated models like Anthropic's Claude 3 Sonnet below, are supported by BedrockLLM.  Hence, we have the more sophisticated ChatBedrock, as shown below.

Also note that the output of ChatBedrock contains much more information than just a text output.

In [16]:
chat_llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN
)

chat_llm.invoke(input="What is the recipe of mayonnaise?")

AIMessage(content='Here is a basic recipe for homemade mayonnaise:\n\nIngredients:\n- 1 egg yolk\n- 1 tablespoon lemon juice or white wine vinegar\n- 1 teaspoon Dijon mustard (optional, for flavor)\n- 1/4 teaspoon salt\n- 3/4 cup neutral oil (like canola, vegetable or grapeseed oil)\n\nInstructions:\n\n1. In a medium bowl, whisk together the egg yolk, lemon juice/vinegar, mustard (if using), and salt.\n\n2. Very slowly, while whisking constantly, drizzle in a few drops of the oil until the mixture begins to thicken and emulsify.\n\n3. Still whisking constantly, drizzle in the remaining oil a few drops at a time until fully incorporated and the mayonnaise is thick.\n\n4. If it gets too thick, you can whisk in a teaspoon or two of warm water to thin it out.\n\n5. Taste and adjust seasoning if needed by adding more lemon juice, vinegar or salt.\n\nThe key is to add the oil very slowly while whisking vigorously to allow a stable emulsion to form. This basic mayo can be flavored with herbs,

## Temperature

Temperature is the thing that gives models creativity.  It controls the randomness of the model's responses.  Setting it to 0.0 (the minimum) typically results in a more deterministic and consistent output while setting it to 1.0 (the maximum) results in more creative responses.  Experiment with the temperature setting in the following cell to see how the output changes as a result.

In [11]:
chat_llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    temperature=0.0
)

chat_llm.invoke(input="What is the recipe of mayonnaise?")

AIMessage(content='Here is a basic recipe for homemade mayonnaise:\n\nIngredients:\n- 1 egg yolk\n- 1 tablespoon lemon juice or white wine vinegar\n- 1/2 teaspoon Dijon mustard (optional)\n- 1/4 teaspoon salt\n- 3/4 cup vegetable oil or mild olive oil\n\nInstructions:\n\n1. In a medium bowl, whisk together the egg yolk, lemon juice/vinegar, mustard (if using), and salt.\n\n2. Very slowly, while whisking constantly, drizzle in the oil just a few drops at a time at first. This allows the mixture to emulsify properly.\n\n3. Once about 1/4 cup of oil has been incorporated, you can start adding the oil in a thin steady stream while whisking vigorously. \n\n4. Continue whisking and adding oil until all the oil is incorporated and the mayonnaise is thick.\n\n5. Taste and adjust seasoning if needed by adding more lemon juice, salt, etc.\n\n6. Transfer to an airtight container and refrigerate for up to 1 week.\n\nThe key is adding the oil very slowly at first to allow the emulsion to form prope

## Limiting the number of tokens returned

The cost of using an LLM is dependent on how many tokens are sent back and forth with the model.  The `max_tokens` parameter can provide a limit on how many total tokens are returned.  Limiting the token count can be useful when you need to ensure that the responses remain concise or when working within strict output size constraints.  Experiment with a few different values for this to see how the output changes.

In [12]:
chat_llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    temperature=0.0,
    max_tokens=10
)

chat_llm.invoke(input="What is the recipe of mayonnaise?")

AIMessage(content='Here is a basic recipe for homemade may', additional_kwargs={'usage': {'prompt_tokens': 16, 'completion_tokens': 10, 'total_tokens': 26}, 'stop_reason': 'max_tokens', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 16, 'completion_tokens': 10, 'total_tokens': 26}, 'stop_reason': 'max_tokens', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-66e34ff9-fad3-4e7a-a496-21bd227ea222-0', usage_metadata={'input_tokens': 16, 'output_tokens': 10, 'total_tokens': 26})